In [16]:
from cpc_pipeline import *

from llm import LLM
gpt35t = LLM("gpt-3.5-turbo")
gpt4 = LLM("gpt-4")

In [23]:
import nltk
import random

max_shift = 5

filtered_corpus = [s for s in nltk.corpus.abc.words() if s.lower() == s]
def make_caesar_cipher(word_length, shift):
    word = random.choice([s for s in filtered_corpus if len(s) == word_length])
    return ''.join([chr((ord(c) + shift - 97) % 26 + 97) for c in word]), word

problem_df = cpc_problems(make_caesar_cipher, {'word_length': [3,4,5], 'shift':range(1, max_shift+1)}, n=1, unwrap_colnames=['problem', 'original'])

Reading cpc_pipeline/make_caesar_cipher3286947891.csv...


In [24]:
prompt = f"This ciphertext is either an anagram or a Caesar cipher. First, assume it's an anagram, and try to unscramble it. If that doesn't work, continue by assuming it's a Caesar cipher and try to decrypt it. Show your work using at least {max_shift} different shifts. "+"Ciphertext: {ciphertext}"
false_start = "Ok, I'll start by treating the ciphertext as an anagram and trying to unscramble it."

def solve_caesar_gpt35t(problem):
    return gpt35t.chat_completion_false_start(prompt.format(ciphertext=problem), false_start=false_start)

def solve_caesar_gpt4(problem):
    return gpt4.chat_completion_false_start(prompt.format(ciphertext=problem), false_start=false_start)

context_df_35t = cpc_contexts(problem_df, solve_caesar_gpt35t)
context_df_4 = cpc_contexts(problem_df, solve_caesar_gpt4)

Creating cpc_pipeline/solve_caesar_gpt35t4090895247.csv...
Creating cpc_pipeline/solve_caesar_gpt42184766945.csv...


In [25]:
context_df_35t['correct'] = context_df_35t.apply(lambda x: 1 if x['original'] in x['context'] else 0, axis=1)
print(np.mean(context_df_35t['correct']))
context_df_4['correct'] = context_df_4.apply(lambda x: 1 if x['original'] in x['context'] else 0, axis=1)
print(np.mean(context_df_4['correct']))

0.4666666666666667
0.4666666666666667


In [26]:
print('\n-----------------\n'.join(context_df_35t['context']))

After trying to unscramble the ciphertext "bsf", it does not seem to form a meaningful word. Let's now try decrypting it using a Caesar cipher with different shifts: 

Shift 1: are
Shift 2: ard
Shift 3: zqc
Shift 4: ypb
Shift 5: xoa

None of the decrypted results form meaningful words either. It looks like the ciphertext may not be a simple Caesar cipher. Would you like me to attempt another type of cipher decoding, or do you have any other clues that could help decrypt it?
-----------------
After trying a few combinations, I was not able to unscramble the ciphertext "vjg" into a meaningful word. Next, I will try to decrypt it using a Caesar cipher with different shifts.
-----------------
After trying different combinations, I couldn't unscramble "dqg" into a meaningful word. Now, I will try decrypting it using a Caesar cipher with different shifts.

1. Shift 1:
Ciphertext: dqg
Plaintext: cpf

2. Shift 2:
Ciphertext: dqg
Plaintext: bpf

3. Shift 3:
Ciphertext: dqg
Plaintext: aoe

4. Sh

In [27]:
print('\n-----------------\n'.join(context_df_4['context']))

The anagram variants for "bsf" could be "sfb", "sbf", "fbs", "fsb", "bsf" and "bfs". However, none of these are recognizable English words.

Continuing with the assumption that it's a Caesar Cipher. Below are the 5 different shifts: 

1. Shift 1: "are"
2. Shift 2: "zdq"
3. Shift 3: "ycp"
4. Shift 4: "xbo"
5. Shift 5: "wan"

Out of these, "are" from the 1st shift seems the most likely decoded message if it's a Caesar Cipher.
-----------------
The given ciphertext "vjg" doesn't appear to form a meaningful word in English when unscrambled, so it's likely not an anagram. Now I'll try assuming it's a Caesar cipher and decrypt it using shifts.

1. Using a shift of 1: "uif"
2. Using a shift of 2: "the"
3. Using a shift of 3: "sgd"
4. Using a shift of 4: "rfc"
5. Using a shift of 5: "qeb"

Among these, "the" is a meaningful word in English, so it's likely that this is the correct decryption with a Caesar cipher using a shift of 2.
-----------------
The letters in "dqg" can be rearranged to cre

In [ ]:
from new_switch_utils import make_non_decreasing

anagram_switching_prompt = "This is a reasoning transcript of an agent trying to decipher some ciphertext. The agent will start treating the ciphertext as an anagram and attempting to unscramble it. The agent may then give up and treat the ciphertext as a Caesar cipher instead. First you will be shown the full transcript, then just a prefix of the transcript. By the end of the prefix transcript, has the agent switched from treating the ciphertext as an anagram to treating it as a Caesar cipher yet?"
def judge_anagram_switching(context, prefix):
        return 1 if gpt4.yesno_completion(anagram_switching_prompt+'\n\nFULL TRANSCRIPT:\n'+context+'\n\nPREFIX TRANSCRIPT:\n'+prefix+"\n\nANSWER:\n") == 'Yes' else 0

def split_and_judge_gpt4(context, chunk_size):
    prefixes = [context[:pref_end] for pref_end in range(chunk_size, len(context)+chunk_size, chunk_size)]
    measured_switching = [judge_anagram_switching(context, prefix) for prefix in prefixes]
    processed_switching = random.choice(make_non_decreasing(measured_switching))
    return prefixes, processed_switching

switching_df_35t = split_and_judge_switching(context_df_35t, split_and_judge_gpt4, 50)
switching_df_4 = split_and_judge_switching(context_df_4, split_and_judge_gpt4, 50)

In [ ]:
anagram_cpc_prompt = """At this point, we're going to stop and consider whether this approach is working or not 
              and leads to a correct solution to the problem being worked on. 
              Otherwise, we should step back and try a different approach."""

from solver import perform_one_token_cpc, perform_cot_cpc
def gpt35t_1t(context):
    return perform_one_token_cpc(gpt35t, context, prompt=anagram_cpc_prompt)
def gpt35t_cot(context):
    return perform_cot_cpc(gpt35t, context, prompt=anagram_cpc_prompt)
def gpt4_1t(context):
    return perform_one_token_cpc(gpt4, context, prompt=anagram_cpc_prompt)
def gpt4_cot(context):
    return perform_cot_cpc(gpt4, context, prompt=anagram_cpc_prompt)

cpc_df_35t = judge_cpc(switching_df_35t, [gpt35t_1t, gpt35t_cot], radius_right=7)
cpc_df_4 = judge_cpc(switching_df_4, [gpt4_1t, gpt4_cot], radius_right=7)

In [ ]:
do_analysis(cpc_df_35t, [gpt35t_1t, gpt35t_cot])
do_analysis(cpc_df_4, [gpt4_1t, gpt4_cot])